In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import torch.optim as optim
from torchvision.utils import save_image

device = torch.device("mps")

In [2]:
print(torch.__version__)

1.14.0.dev20221115


In [3]:
bs = 64
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [4]:
class Encoder(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(Encoder, self).__init__()
        self.lin1 = nn.Linear(x_dim, h_dim1)
        self.lin2 = nn.Linear(h_dim1, h_dim2)
        self.lin31 = nn.Linear(h_dim2, z_dim)
        self.lin32 = nn.Linear(h_dim2, z_dim)
        
    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        return self.lin31(x), self.lin32(x) # mu, log_var
    

In [5]:
class Decoder(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(Decoder, self).__init__()
        self.lin4 = nn.Linear(z_dim, h_dim2)
        self.lin5 = nn.Linear(h_dim2, h_dim1)
        self.lin6 = nn.Linear(h_dim1, x_dim)

    def forward(self, z):
        h = F.relu(self.lin4(z))
        h = F.relu(self.lin5(h))
        return F.sigmoid(self.lin6(h)) 

In [7]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        self.encoder = Encoder(x_dim, h_dim1, h_dim2, z_dim)
        self.decoder = Decoder(x_dim, h_dim1, h_dim2, z_dim)
        
    def sampling(self, mu, log_var): #TODO
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) 

    def forward(self, x):
        mu, log_var = self.encoder.forward(x)
        z = self.sampling(mu, log_var)
        return self.decoder.forward(z), mu, log_var
    
    def generate(self, z):
        return self.decoder.forward(z)
    
    #def loss_function(recon_x, x, mu, log_var):
    #    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    #    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    #    return BCE + KLD
    
    #def train(self):
    #    optimizer = optim.Adam(self.parameters())

In [8]:
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
vae = vae.to(device)
optimizer = optim.Adam(vae.parameters())

In [9]:
def loss_function(recon_x, x, mu, log_var):
        BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
        KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        return BCE + KLD

In [27]:
for i in train_loader:
    print(i)

TypeError: can't convert mps:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [10]:
def train(epoch):
    vae.train()
    train_loss = 0
    batch_idx = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        data = data.reshape([-1, 784]) / 256
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae.forward(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        batch_idx += 1
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [13]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.to(device)
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [12]:
for epoch in range(1, 50):
    train(epoch)
    #test()

Train Epoch: 1 [3200/60000 (5%)]	Loss: 130.819931
Train Epoch: 1 [6400/60000 (11%)]	Loss: 157.259003
Train Epoch: 1 [9600/60000 (16%)]	Loss: 142.721939
Train Epoch: 1 [12800/60000 (21%)]	Loss: 141.639603
Train Epoch: 1 [16000/60000 (27%)]	Loss: 134.944687
Train Epoch: 1 [19200/60000 (32%)]	Loss: 138.525665
Train Epoch: 1 [22400/60000 (37%)]	Loss: 152.466797
Train Epoch: 1 [25600/60000 (43%)]	Loss: 142.369064
Train Epoch: 1 [28800/60000 (48%)]	Loss: 148.463943
Train Epoch: 1 [32000/60000 (53%)]	Loss: 148.783905
Train Epoch: 1 [35200/60000 (59%)]	Loss: 153.288193
Train Epoch: 1 [38400/60000 (64%)]	Loss: 135.021530
Train Epoch: 1 [41600/60000 (69%)]	Loss: 142.497391
Train Epoch: 1 [44800/60000 (75%)]	Loss: 132.742340
Train Epoch: 1 [48000/60000 (80%)]	Loss: 147.959717
Train Epoch: 1 [51200/60000 (85%)]	Loss: 146.646393
Train Epoch: 1 [54400/60000 (91%)]	Loss: 121.645309
Train Epoch: 1 [57600/60000 (96%)]	Loss: 136.257645
====> Epoch: 1 Average loss: 141.3232
Train Epoch: 2 [3200/60000 (5%

Train Epoch: 9 [32000/60000 (53%)]	Loss: 129.160294
Train Epoch: 9 [35200/60000 (59%)]	Loss: 139.813614
Train Epoch: 9 [38400/60000 (64%)]	Loss: 132.889999
Train Epoch: 9 [41600/60000 (69%)]	Loss: 139.789352
Train Epoch: 9 [44800/60000 (75%)]	Loss: 143.650513
Train Epoch: 9 [48000/60000 (80%)]	Loss: 127.142006
Train Epoch: 9 [51200/60000 (85%)]	Loss: 141.384064
Train Epoch: 9 [54400/60000 (91%)]	Loss: 140.332382
Train Epoch: 9 [57600/60000 (96%)]	Loss: 143.777069
====> Epoch: 9 Average loss: 139.2173
Train Epoch: 10 [3200/60000 (5%)]	Loss: 139.787628
Train Epoch: 10 [6400/60000 (11%)]	Loss: 127.650291
Train Epoch: 10 [9600/60000 (16%)]	Loss: 150.697433
Train Epoch: 10 [12800/60000 (21%)]	Loss: 149.689697
Train Epoch: 10 [16000/60000 (27%)]	Loss: 141.082016
Train Epoch: 10 [19200/60000 (32%)]	Loss: 145.172989
Train Epoch: 10 [22400/60000 (37%)]	Loss: 138.385223
Train Epoch: 10 [25600/60000 (43%)]	Loss: 142.764618
Train Epoch: 10 [28800/60000 (48%)]	Loss: 143.905701
Train Epoch: 10 [3200

Train Epoch: 17 [51200/60000 (85%)]	Loss: 128.698120
Train Epoch: 17 [54400/60000 (91%)]	Loss: 128.074661
Train Epoch: 17 [57600/60000 (96%)]	Loss: 128.402222
====> Epoch: 17 Average loss: 138.6728
Train Epoch: 18 [3200/60000 (5%)]	Loss: 145.498581
Train Epoch: 18 [6400/60000 (11%)]	Loss: 144.693604
Train Epoch: 18 [9600/60000 (16%)]	Loss: 132.795425
Train Epoch: 18 [12800/60000 (21%)]	Loss: 132.824402
Train Epoch: 18 [16000/60000 (27%)]	Loss: 133.990509
Train Epoch: 18 [19200/60000 (32%)]	Loss: 150.167099
Train Epoch: 18 [22400/60000 (37%)]	Loss: 136.277054
Train Epoch: 18 [25600/60000 (43%)]	Loss: 140.354294
Train Epoch: 18 [28800/60000 (48%)]	Loss: 140.065521
Train Epoch: 18 [32000/60000 (53%)]	Loss: 135.542664
Train Epoch: 18 [35200/60000 (59%)]	Loss: 144.819839
Train Epoch: 18 [38400/60000 (64%)]	Loss: 124.440399
Train Epoch: 18 [41600/60000 (69%)]	Loss: 142.085571
Train Epoch: 18 [44800/60000 (75%)]	Loss: 145.236801
Train Epoch: 18 [48000/60000 (80%)]	Loss: 138.367523
Train Epoch

Train Epoch: 26 [9600/60000 (16%)]	Loss: 140.310242
Train Epoch: 26 [12800/60000 (21%)]	Loss: 138.684662
Train Epoch: 26 [16000/60000 (27%)]	Loss: 131.169495
Train Epoch: 26 [19200/60000 (32%)]	Loss: 147.331116
Train Epoch: 26 [22400/60000 (37%)]	Loss: 124.976547
Train Epoch: 26 [25600/60000 (43%)]	Loss: 139.301666
Train Epoch: 26 [28800/60000 (48%)]	Loss: 134.490936
Train Epoch: 26 [32000/60000 (53%)]	Loss: 149.117401
Train Epoch: 26 [35200/60000 (59%)]	Loss: 137.342972
Train Epoch: 26 [38400/60000 (64%)]	Loss: 151.054688
Train Epoch: 26 [41600/60000 (69%)]	Loss: 143.097137
Train Epoch: 26 [44800/60000 (75%)]	Loss: 132.022232
Train Epoch: 26 [48000/60000 (80%)]	Loss: 146.288757
Train Epoch: 26 [51200/60000 (85%)]	Loss: 142.716644
Train Epoch: 26 [54400/60000 (91%)]	Loss: 145.077698
Train Epoch: 26 [57600/60000 (96%)]	Loss: 133.320709
====> Epoch: 26 Average loss: 137.3644
Train Epoch: 27 [3200/60000 (5%)]	Loss: 124.472115
Train Epoch: 27 [6400/60000 (11%)]	Loss: 140.547134
Train Epoch

Train Epoch: 34 [28800/60000 (48%)]	Loss: 133.381851
Train Epoch: 34 [32000/60000 (53%)]	Loss: 134.587067
Train Epoch: 34 [35200/60000 (59%)]	Loss: 158.078918
Train Epoch: 34 [38400/60000 (64%)]	Loss: 127.089897
Train Epoch: 34 [41600/60000 (69%)]	Loss: 143.241074
Train Epoch: 34 [44800/60000 (75%)]	Loss: 132.167236
Train Epoch: 34 [48000/60000 (80%)]	Loss: 138.885361
Train Epoch: 34 [51200/60000 (85%)]	Loss: 145.309036
Train Epoch: 34 [54400/60000 (91%)]	Loss: 132.902420
Train Epoch: 34 [57600/60000 (96%)]	Loss: 137.034943
====> Epoch: 34 Average loss: 136.6265
Train Epoch: 35 [3200/60000 (5%)]	Loss: 147.213226
Train Epoch: 35 [6400/60000 (11%)]	Loss: 133.896454
Train Epoch: 35 [9600/60000 (16%)]	Loss: 133.970230
Train Epoch: 35 [12800/60000 (21%)]	Loss: 126.693932
Train Epoch: 35 [16000/60000 (27%)]	Loss: 129.267578
Train Epoch: 35 [19200/60000 (32%)]	Loss: 149.793091
Train Epoch: 35 [22400/60000 (37%)]	Loss: 142.986069
Train Epoch: 35 [25600/60000 (43%)]	Loss: 128.552521
Train Epoch

Train Epoch: 42 [48000/60000 (80%)]	Loss: 141.722168
Train Epoch: 42 [51200/60000 (85%)]	Loss: 131.787369
Train Epoch: 42 [54400/60000 (91%)]	Loss: 140.036255
Train Epoch: 42 [57600/60000 (96%)]	Loss: 124.031517
====> Epoch: 42 Average loss: 136.2835
Train Epoch: 43 [3200/60000 (5%)]	Loss: 142.429291
Train Epoch: 43 [6400/60000 (11%)]	Loss: 138.438126
Train Epoch: 43 [9600/60000 (16%)]	Loss: 138.916473
Train Epoch: 43 [12800/60000 (21%)]	Loss: 150.721313
Train Epoch: 43 [16000/60000 (27%)]	Loss: 129.241974
Train Epoch: 43 [19200/60000 (32%)]	Loss: 124.563271
Train Epoch: 43 [22400/60000 (37%)]	Loss: 135.393204
Train Epoch: 43 [25600/60000 (43%)]	Loss: 133.367828
Train Epoch: 43 [28800/60000 (48%)]	Loss: 140.380966
Train Epoch: 43 [32000/60000 (53%)]	Loss: 143.743881
Train Epoch: 43 [35200/60000 (59%)]	Loss: 125.020256
Train Epoch: 43 [38400/60000 (64%)]	Loss: 138.988129
Train Epoch: 43 [41600/60000 (69%)]	Loss: 135.430130
Train Epoch: 43 [44800/60000 (75%)]	Loss: 144.229477
Train Epoch

In [15]:
with torch.no_grad():
    z = torch.randn(64, 2).to(device)
    sample = vae.generate(z).to(device)
    
    save_image(sample.view(64, 1, 28, 28), './samples/sample_' + '.png')